# Age and Emotion Detection for Movie Theatre

This notebook contains **only the GUI and logic** for the **Age and Emotion Detection** task.

### Description
- Detect faces in real-time using webcam.
- Predict **age** of each person:
    - If age < 13 or > 60: Mark with red rectangle and display **"Not allowed"**.
    - If 13 <= age <= 60: Predict **emotion** and display.
- Save all results (age, emotion, entry time) to a CSV file.

### Guidelines
- Replace dummy models with your own trained models for better accuracy.
- Run the GUI using Streamlit for an interactive experience.


## 1. Requirements

Ensure you have the following installed:

```bash
pip install opencv-python mediapipe tensorflow scikit-learn pandas streamlit
```


In [ ]:
import streamlit as st
import cv2
import numpy as np
import pandas as pd
import os
from datetime import datetime
from keras.models import load_model

# -------------------------
# Placeholder: Load your trained models here
# -------------------------
# Replace with your actual models
# Example: age_model = load_model("age_model.h5")
# Example: emotion_model = load_model("emotion_model.h5")

def dummy_age_predict(face_img):
    # Dummy: randomly predict an age between 5 and 80
    return np.random.randint(5, 80)

def dummy_emotion_predict(face_img):
    # Dummy: randomly pick an emotion
    emotions = ["Happy", "Sad", "Scared", "Neutral"]
    return np.random.choice(emotions)

# -------------------------
# Utility: Save results
# -------------------------
LOG_FILE = "movie_theatre_log.csv"
if not os.path.exists(LOG_FILE):
    pd.DataFrame(columns=["Age", "Emotion", "Time"]).to_csv(LOG_FILE, index=False)

def log_entry(age, emotion):
    df = pd.read_csv(LOG_FILE)
    df.loc[len(df)] = [age, emotion, datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
    df.to_csv(LOG_FILE, index=False)

# -------------------------
# Streamlit GUI
# -------------------------
st.title("Age and Emotion Detection for Movie Theatre")

run = st.checkbox("Start Webcam")

FRAME_WINDOW = st.image([])
cap = None

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

if run:
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            st.error("Failed to capture video")
            break
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            age = dummy_age_predict(face_img)
            if age < 13 or age > 60:
                label = f"Age: {age} - Not allowed"
                color = (0, 0, 255)
                emotion = "-"
            else:
                emotion = dummy_emotion_predict(face_img)
                label = f"Age: {age}, Emotion: {emotion}"
                color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            log_entry(age, emotion)
        FRAME_WINDOW.image(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
else:
    st.info("Check 'Start Webcam' to begin detection.")

if cap:
    cap.release()
